In [ ]:
!pip install -q category_encoders xfeat texthero

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from scipy.optimize import minimize
import pickle

from sklearn.model_selection import StratifiedKFold
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

import re
import os
import torch
import string
import random
import warnings

import category_encoders as ce
import xfeat
import texthero as hero
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import QuantileTransformer

warnings.filterwarnings('ignore')

In [ ]:
def seed_all(seed):
  random.seed(seed)
  np.random.seed(seed)
  os.environ["PYTHONHASHSEED"] = str(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed_all(seed)
  torch.backends.cudnn.daterministic = True

SEED = 42
seed_all(SEED)

In [ ]:
DATA_PATH = "./data/"
train = pd.read_csv(DATA_PATH + 'train.csv')
test = pd.read_csv(DATA_PATH + 'test.csv')

In [ ]:
TRAIN_LEN = len(train)
FOLDS = 5
NAME = "baseline001"

BASE_DIR_MLP = "./pretrained_models/mlp/"
BASE_DIR_MLP2 = "./pretrained_models/mlp_2/"
MODEL_DIR_MLP = BASE_DIR_MLP2 + "models/"
DATA_DIR_MLP = BASE_DIR_MLP + "data/"

In [ ]:
train_x = pickle.load(open(DATA_DIR_MLP + "train_x.pkl", 'rb'))
test_x = pickle.load(open(DATA_DIR_MLP + "test_x.pkl", 'rb'))

train_y = train["state"]

In [ ]:
def make_skf(train_x, train_y, random_state=2021):
    skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=random_state)
    folds_idx = [(t, v) for (t, v) in skf.split(train_x, train_y)]
    return folds_idx


def threshold_optimization(y_true, y_pred, metrics=None):
    def f1_opt(x):
        if metrics is not None:
            score = -metrics(y_true, y_pred >= x)
        else:
            raise NotImplementedError
        return score
    result = minimize(f1_opt, x0=np.array([0.5]), method='Nelder-Mead')
    best_threshold = result['x'].item()
    return best_threshold


def optimized_f1(y_true, y_pred):
    bt = threshold_optimization(y_true, y_pred, metrics=f1_score)
    score = f1_score(y_true, y_pred >= bt)
    return score    

In [ ]:
class MyLogisticRegressionModel:
    def __init__(self, name=None, params=None, fold=None, train_x=None, train_y=None, test_x=None, metrics=None, seeds=None):
        self.train_x = train_x
        self.train_y = train_y
        self.test_x = test_x
        self.name = name
        self.params = params
        self.metrics = metrics  
        self.kfold = fold  
        self.oof = None
        self.preds = None
        self.seeds = seeds if seeds is not None else [2020] 
        self.models = {}  

    def build_model(self):
        model = LogisticRegression(**self.params)
        return model              

    def predict_cv(self, pretrained=False, model_dir=MODEL_DIR_MLP):
        oof_seeds = []
        scores_seeds = []
        for seed in self.seeds:
            oof = []
            va_idxes = []
            scores = []
            train_x = self.train_x.values
            train_y = self.train_y.values
            fold_idx = self.kfold(self.train_x, self.train_y, random_state=seed) 

            for cv_num, (tr_idx, va_idx) in enumerate(fold_idx):
                tr_x, va_x = train_x[tr_idx], train_x[va_idx]
                tr_y, va_y = train_y[tr_idx], train_y[va_idx]
                va_idxes.append(va_idx)
                model = self.build_model()

                model_name = f"{self.name}_SEED{seed}_FOLD{cv_num}_model.pkl"
                model_path = model_dir + model_name
    
                if pretrained == False:
                  model.fit(tr_x, tr_y)                            
                  pickle.dump(model, open(model_name, 'wb'))
                else:
                  model = pickle.load(open(model_path, 'rb'))
                
                self.models[model_name] = model  
                
                pred = model.predict_proba(va_x)[:, 1]
                oof.append(pred)

                score = self.get_score(va_y, pred)
                scores.append(score)
                print(f"SEED:{seed}, FOLD:{cv_num} =====> val_score:{score}")

            va_idxes = np.concatenate(va_idxes)
            oof = np.concatenate(oof)
            order = np.argsort(va_idxes)
            oof = oof[order]
            oof_seeds.append(oof)
            scores_seeds.append(np.mean(scores))
            
        oof = np.mean(oof_seeds, axis=0)
        self.oof = oof
        print(f"model:{self.name} score:{self.get_score(self.train_y, oof)}\n")
        return oof      

    def inference(self, pretrained=False, model_dir=MODEL_DIR_MLP):
        preds_seeds = []
        for seed in self.seeds:
            preds = []
            test_x = self.test_x.values
            for cv_num in range(FOLDS):
                print(f"-INFERENCE- SEED:{seed}, FOLD:{cv_num}")
                model_name = f"{self.name}_SEED{seed}_FOLD{cv_num}_model.pkl"
                model_path = model_dir + model_name
                if pretrained == False:
                  model = self.models[model_name]                
                else:
                  model = pickle.load(open(model_path, 'rb'))
                pred = model.predict_proba(test_x)[:, 1]
                preds.append(pred)
            preds = np.mean(preds, axis=0)
            preds_seeds.append(preds)
        preds = np.mean(preds_seeds, axis=0)
        self.preds = preds
        return preds  
    
    def get_score(self, y_true, y_pred):
        score = self.metrics(y_true, y_pred)
        return score


In [ ]:
params = {
    'penalty': 'l2',
    'dual': False,
    'tol': 1e-4,
    'C': 1.0,
    'fit_intercept': True,
    'intercept_scaling': 1,
    'class_weight': None,
    'random_state': 2021,
    'solver': 'lbfgs',
    'max_iter': 100,
    'multi_class': 'auto',
    'verbose': 0,
    'warm_start': False,
    'n_jobs': -1, 
    'l1_ratio': None
}

model = MyLogisticRegressionModel(name=NAME, 
                    params=params,
                    fold=make_skf,
                    train_x = train_x,
                    train_y=train_y,
                    test_x = test_x,
                    metrics=optimized_f1, 
                    seeds=[0, 1, 2]
                   )

oof = model.predict_cv()  
preds = model.inference()

best_threshold = threshold_optimization(y_true=train_y, y_pred=oof, metrics=f1_score) 
print(f"best_threshold is {best_threshold}\n")

labels = preds >= best_threshold